## Block 1

In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

#Download des Shakespeare-Texts ins Skript
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

#Nur Ausschnitt des Texts wird verwendet
text = text[300000:800000]

## Block 2

In [3]:
# Array, das alle im Text vorkommenden Buchstaben und Zeichen sortiert enthält
characters = sorted(set(text))
print(len(characters))
# Dictionary, das alle Buchstaben/Zeichen sortiert enthält, die jeweils auf einen Index verweisen
char_to_index = dict((c, i) for i, c in enumerate(characters))
# Buchstaben/Zeichen dictionary umgedreht - Index weist auf Zahl
index_to_char = dict((i, c) for i, c in enumerate(characters))


39


## Block 3

In [4]:

# SEQ_LENGTH bestimmt die Länge, die ein Satz, der bearbeitet wird, hat
SEQ_LENGTH = 40
# Step_Size bestimmt den Shift, der zwischen den Sequenzen liegt
# Je geringer die STEP_SIZE, desto mehr Trainingssequenzen werden verwendet
# Diese überlappen sich dann aber umso mehr
STEP_SIZE = 3

# In das Array sentences werden alle Sequenzen (Anzahl: (len(text)/SEQ_LENGTH)) gespeichert
sentences = []
# Das next_char Array enthält jeweils das nächste Zeichen, das nach dem Satz, der
# über den selben Index im sentences Array gespeichert ist, im Text kommt
next_char = []
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

# Block 4

In [5]:
# x sind unsere Eingabedaten 
# y enthält unsere Zieldaten
# Diese werden dem LSTM zum Trainieren "gefüttert"
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

# Block 5

In [7]:
# Unser Modell ist ein Long Short-Term Memory
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=5)

Epoch 1/5
651/651 [==============================] - 38s 56ms/step - loss: 2.1867
Epoch 2/5
651/651 [==============================] - 43s 66ms/step - loss: 1.7401
Epoch 3/5
651/651 [==============================] - 45s 69ms/step - loss: 1.6018
Epoch 4/5
651/651 [==============================] - 54s 84ms/step - loss: 1.5286
Epoch 5/5
651/651 [==============================] - 53s 82ms/step - loss: 1.4812


## Block 6

Aufgabe

In [8]:
# Der sample()-Funktion wird ein Array mit Wahrscheinlichkeiten von Zeichen, die als nächstes
# im zu generierenden Text verwendet werden können und der sogenannte temperature-Wert übergeben
# Je niedriger der Temperaturwert, desto "realistischer" wird der Text
# Je höher der Temperaturwert, desto kreativer aber auch "falscher" kann der Text werden

# Funktionen für Aufgabe:
# np.log(x)
# np.exp(x)
# np.sum(x)
# np.random.multinomial(1, x, 1)

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    # Die folgenden Anweisungen sorgen dafür, dass verhältnismäßig geringe Wahrscheinlichkeiten
    # noch geringer und verhältnismäßig hohe Wahrscheinlichkeiten noch höher werden
    # Bei temperature = 1 bleiben die Wahrscheinlichkeiten gleich
    # logarithmieren 
    preds =
    # durch temperature teilen
    preds =
    # exponieren
    exp_preds =
    # exponierte predictions/summe exponierte predictions
    preds =
    # "Zufällige" Wahl einer der Buchstaben
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Block 7

In [9]:
def generate_text(length, temperature):
    # Startindex wird so gewählt, dass nicht das Ende des Texts erreicht wird
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    # Die ersten 40 Zeichen werden aus dem Text genommen
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    # Schleife läuft so oft durch, wie die Länge, die übergeben wurde
    for i in range(length):
        
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            # x_predictions sind die Eingabedaten auf Basis der aktuellen Sequenz,
            # die dem Model gegeben werden   
            x_predictions[0, t, char_to_index[char]] = 1
        # Das Model gibt predictions für alle (39) Zeichen wieder
        predictions = model.predict(x_predictions, verbose=0)[0]
        # sample-Funktion "wählt" den nächsten Buchstaben aus
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]
        # nächster Buchstabe wird angehängt
        generated += next_character
        sentence = sentence[1:] + next_character    
    return generated

In [10]:
print('temp: 0.5')
print(generate_text(300, 0.5))
print('temp: 1')
print(generate_text(300, 1))
print('temp: 1.5')
print(generate_text(300, 1.5))


temp: 0.1
d queen;
and would by combat make her good men the words,
that i have the sent and the seek the broke
that the seek the sent and the sent a son,
and the days the sent and the words with the words,
and the sent the sentence to the way the words,
and the words with the words in the words,
and the sent and the words in the words,
and with th
temp: 0.5
out
betwixt the prince and beggar: i have said
that the way hath you that is my death couns,
and see his heads of my braves and brave
and the sadd to the words, where you say,
the back to right was the truth would him north,
and the shalt that i slard with the words.

yerford:
is my faith, on shalt his brother of the prince the week.

fri
temp: 1
g!
doth set my pugging tooth on edge;
for which was swiff, each his shears, or,
and tenple skinch touth her! 
belove:
spreeves gage sial of this that i cortels!

duke of york:
so, o'ergaterapteaty brokes or with
that to thee of remeign.

rouses:
sims should becoury with his reath;
for my h